In [5]:
import os
os.chdir("/Users/g.frigo/repo/py_ab/src/")

from pyab_experiment.language.grammar import ExperimentAST, ExperimentConditional,ExperimentGroup,ConditionalType
from pyab_experiment.parser import parse

file_path = "/Users/g.frigo/repo/py_ab/src/tests/unit/test_programs/full_grammar.pyab"
#file_path = "/Users/g.frigo/repo/py_ab/src/tests/unit/test_programs/basic_experiment.pyab"
#file_path = "/Users/g.frigo/repo/py_ab/src/tests/unit/test_programs/conditional.pyab"
with open(file_path,'r') as fp:
    ast = parse(fp.read())

In [6]:
class PythonCodeGen:
    def __init__(self,experiment_ast:ExperimentAST,indentation_char:str="->"):
        self._experiment_ast = experiment_ast
        self._local_vars = set()
        self._splitting_vars = set()
        self._conditional_ids = set() #to save conditional variables seen
        self._indentation_char = indentation_char
        self._newline = "\n"
        self._indent_depth = 0


    def indent(self)->str:
        return ''.join([self._indentation_char*self._indent_depth])

    def generate(self)->str:
        docstring_header = f'# Autogenerated by pyab. Do not modify manually{self._newline}'
        self._indent_depth+=1
        salt_def = f"{self.indent()}salt='{self._experiment_ast.salt}'{self._newline}" if self._experiment_ast.salt is not None else ""

        if self._experiment_ast.splitting_fields:
            for var in self._experiment_ast.splitting_fields:
                self._local_vars.add(var)

        fn_body = self._generate_conditionals(self._experiment_ast.conditions)


        fn_defn = f"def {self._experiment_ast.id}({', '.join(self._local_vars)}):{self._newline}"
        return f"{docstring_header}{fn_defn}{salt_def}{fn_body}"

    def _generate_conditionals(self,condition:ExperimentConditional|list[ExperimentGroup])->str:
        """generates the (possibly nested) conditional statements, and their return functions
            goes through through all the contitionals and rendering
            appropriate function definitions
        """
        match condition:
            case ExperimentConditional():
                predicate = "some_condition" #condition.predicate
                self._indent_depth+=1
                true_branch_stmt = self._generate_conditionals(condition.true_branch)
                self._indent_depth-=1
                false_branch_stmt = self._generate_conditionals(condition.false_branch) if condition.false_branch is not None else ""
                match condition.conditional_type:
                    case ConditionalType.IF:
                        return f"{self.indent()}if {predicate}:{self._newline}{true_branch_stmt}{false_branch_stmt}"
                    case ConditionalType.ELIF:
                        return f"{self.indent()}elif {predicate}:{self._newline}{true_branch_stmt}{false_branch_stmt}"
                    case ConditionalType.ELSE:
                        return f"{self.indent()}else:{self._newline}{true_branch_stmt}{false_branch_stmt}"

            case [*_]:
                statement = self._generate_group_return_statement(condition)
                return statement

            case _:
                raise Exception("boom")


    def _generate_group_return_statement(self,group_statement:list[ExperimentGroup])->str:
        """unwrap the experiment group, into a partial function call that applies the splitter logic
        """

        population_list = str([group.group_definition for group in group_statement])
        weight_list = str([group.group_weight for group in group_statement])
        return f"{self.indent()}return partial(deterministic_choice,population={population_list},weights={weight_list}){self._newline}"





In [7]:

generator = PythonCodeGen(ast)
print(generator.generate())


# Autogenerated by pyab. Do not modify manually
def complex_experiment_defn(my_fld_1, my_fld):
->salt='csdvs887'
->if some_condition:
->->if some_condition:
->->->return partial(deterministic_choice,population=[123.0, 9.3, 'abc'],weights=[3.4, 5.0, 3.0])
->->elif some_condition:
->->->return partial(deterministic_choice,population=['Setting 1.1.1', 'Setting 1.1.2'],weights=[1.0, 0.0])
->->elif some_condition:
->->->return partial(deterministic_choice,population=['Setting 1.2.1', 'Setting 1.2.2'],weights=[0.5, 0.5])
->->else:
->->->return partial(deterministic_choice,population=['Setting 1.3.1', 'Setting 1.3.2'],weights=[0.5, 0.5])
->else:
->->return partial(deterministic_choice,population=['default'],weights=[1.0])



In [8]:
def get_experiment()->Experiment_groups:
    ...
def choose_group()->

def find_slice(settings: Experiment, **kwargs) -> list[ExperimentGroup]: #router

def demux_groups(settings: Experiment, payload: list[Routable]) -> dict[str, list[Routable]]:
        groups = find_slice(settings, input)
        population = [group.group_definition for group in groups]
        weights = [group.group_weight for group in groups]

        # build key to hash
        key = "".join(
            [str(input.get(field, "")) for field in settings.splitting_fields]
        )

        if len(key) == 0:
            # random choice
            group = deterministic_choice(None, population, weights)
        else:
            key = key + (settings.salt or "")
            group = deterministic_choice(key, population, weights)


SyntaxError: expected ':' (2313725062.py, line 3)